# Elasticsearch test for jm_project

Ce notebook teste la possibilité d'uploader les descriptions d'offres d'emploi sur elasticsearch, et fait test de query de la base de donnée.

Ce notebook suppose qu'on a lancé le docker-compose d'elasticsearch créé dans le cours de datascientest

In [2]:
import json
from elasticsearch import Elasticsearch, helpers

/Users/davidrosenfeld/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [54]:
jobs_path = 'data/adzuna_jobs_2023-11-21-13_51_47.json'

Read in scraped job vacancies:

In [5]:
with open(jobs_path, 'r') as read_file:
     jobs = json.load(read_file)

Keep only the id and description from the data:

In [11]:
def filter_keys(input_dict):
    return {key: input_dict[key] for key in ['id', 'description']}

In [12]:
jobs_filtered = [filter_keys(item) for item in jobs]

In [15]:
# Connexion au cluster
es = Elasticsearch(hosts = "http://@localhost:9200")

In [55]:
# Si il existe déjà, la ligne suivant peut êter utilisée pour supprimer l'index:
# es.indices.delete(index='jm_test')

Creation de l'index, nom donné: `jm_test`

In [25]:
es.indices.create(index = 'jm_test')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'jm_test'})

Upload des données à l'index:

In [27]:
helpers.bulk(es, jobs_filtered, index = 'jm_test')

(5300, [])

Création d'une fonction qui donne le nombre d'offres d'emplois dont la description contient un mot-clé:

In [52]:
def query_index(query_term: str,
                index_name: str = 'jm_test'):
    """
    Fonction pour chercher le nombre d'offres d'emplois contenant la string query_term dans les descriptions 
    d'offres d'emploi de notre base de donnée elasticsearch
    """
    my_query = {
        "match": {
            "description": {
                "query": query_term
                }
        }
    }

    resp = es.search(index = index_name, query = my_query)

    return resp['hits']['total']['value']

In [53]:
query_index(query_term = 'python')

267